In [8]:
!pip install transformers

     |████████████████████████████████| 778kB 8.2MB/s 
     |████████████████████████████████| 890kB 21.8MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
     |████████████████████████████████| 1.1MB 58.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=30f075aac1a581e858197c60050010a77be12557aa11a67c72017e8c0e1c42cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [9]:
import torch
import json
import csv
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


# Load GPT-2 Medium 

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', model_max_length=1024)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Helper functions

In [23]:
class ModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def generate(self, prompt, keywords, category):
        input = '<|startoftext|>'
        if category:
            assert category in ['bad', 'moderate', 'good']
            input += f'~`{category}'
        if keywords:
            keywords = [ k.replace(' ', '-') for k in keywords ]
            input += f"~^{' '.join(keywords)}"
        
        input += f"~@{prompt if prompt else ''}"
        input_encoded = self.tokenizer.encode(input, return_tensors='pt').to(self.device)

        # TODO: make these settings adjustable
        outputs = self.model.generate(
            input_encoded,
            do_sample=True, 
            max_length=600, 
            top_k=30, 
            top_p=0.96, 
            num_return_sequences=3
        )

        # TODO: select outputs with keywords (?)
        outputs_decoded = [ self.tokenizer.decode(out, skip_special_tokens=True) for out in outputs ]
        return outputs_decoded

# testM = ModelWrapper('s', 3)
# testM.generate('ff', ['sex', 'bad thing'], 'good')


# Reviews dataset

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
DATA_PATH = '/content/drive/My Drive/Data Science/Datasets'
!ls '$DATA_PATH'

reviews_nlp_encoded.txt


In [13]:
from torch.utils.data import Dataset, DataLoader

class ReviewsDataset(Dataset):
    def __init__(self, filename):
        super().__init__()

        with open(filename) as data_file:
          self.reviews = data_file.readlines()
        
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        return self.reviews[idx].replace('\n', '')

review_dataset = ReviewsDataset(f'{DATA_PATH}/reviews_nlp_encoded.txt')
review_loader = DataLoader(review_dataset, batch_size=1, shuffle=True)

# Hyperparameters

In [17]:
BATCH_SIZE = 10
EPOCHS = 7
LEARNING_RATE = 1e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 1000
from transformers import AdamW, get_linear_schedule_with_warmup

# Training

In [18]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0

loss_history = []
batch_count = 0
loss_every = 100

temp_reviews_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    print(f"EPOCH {epoch+1}")
    epoch_loss_history = []
  
    for review in review_loader:
        # fit as many review sequences into MAX_SEQ_LEN sequence as possible
        review_tens = torch.tensor(tokenizer.encode(review[0], max_length=MAX_SEQ_LEN, truncation=True)).unsqueeze(0).to(device)
        
        # work_reviews_tens = review_tens

        # the first review sequence in the sequence
        if not torch.is_tensor(temp_reviews_tens):
            temp_reviews_tens = review_tens
            continue
        else:
            # the next review does not fit in so we process the sequence and leave the last review 
            # as the start for next sequence 
            if temp_reviews_tens.size()[1] + review_tens.size()[1] < MAX_SEQ_LEN:
                # add the review to sequence, continue and try to add more
                temp_reviews_tens = torch.cat([temp_reviews_tens, review_tens[:,1:]], dim=1)
                continue
            else:
                work_reviews_tens, temp_reviews_tens = temp_reviews_tens, review_tens
                

        # sequence ready, process it trough the model
        outputs = model(work_reviews_tens, labels=work_reviews_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        epoch_loss_history.append(loss.detach().item())
                       
        proc_seq_count += 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == loss_every:
            batch_count = 0
            avg_loss = np.array(epoch_loss_history)[-loss_every*BATCH_SIZE+1:].mean()
            print(f'Avg. loss: {avg_loss:.4f}')
    
    loss_history.append(torch.tensor(epoch_loss_history))
    # store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_small_review_{epoch+1}.pt"))

EPOCH 1
Avg. loss: 3.8925
Avg. loss: 3.7627
Avg. loss: 3.6264
Avg. loss: 3.5239
EPOCH 2
Avg. loss: 3.4120
Avg. loss: 3.3847
Avg. loss: 3.3262
Avg. loss: 3.2752
Avg. loss: 3.2462
EPOCH 3
Avg. loss: 3.2140
Avg. loss: 3.1862
Avg. loss: 3.1637
Avg. loss: 3.1520
EPOCH 4
Avg. loss: 3.1195
Avg. loss: 3.1300
Avg. loss: 3.1064
Avg. loss: 3.1225
Avg. loss: 3.0935
EPOCH 5
Avg. loss: 3.0671
Avg. loss: 3.0814
Avg. loss: 3.0594
Avg. loss: 3.0678
Avg. loss: 3.0427
EPOCH 6
Avg. loss: 3.0398
Avg. loss: 3.0294
Avg. loss: 3.0355
Avg. loss: 3.0137
EPOCH 7
Avg. loss: 2.9884
Avg. loss: 3.0057
Avg. loss: 2.9942
Avg. loss: 2.9796
Avg. loss: 3.0018


# Load model

In [19]:
!ls 

drive  sample_data  trained_models


In [34]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)
model.load_state_dict(torch.load('trained_models/gpt2_small_review_6.pt'))
model.eval()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [35]:
wrapped_model = ModelWrapper(model, tokenizer)

In [39]:
wrapped_model.generate('', ['boring'], 'bad')

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


["<|startoftext|>~`bad~^boring~@Loved this book. Lots of detail in the book, and an awful lot of dialogue. But overall was a great read. This was one of those books that I'm looking forward to reading in the future.",
 "<|startoftext|>~`bad~^boring~@This review was really interesting to read. It's not for everybody. It is a bit annoying, but it makes for good reading when you are bored with boring reading. I read it because I was bored of the characters and the boring parts. There were a few issues I can discuss, but I will just say that the characters were annoying and I can't go into more detail, and some of the descriptions are lame and confusing. There were some moments where I felt like the whole story was over, like the ending. The ending isn't even close, and I feel like it might have been better left to another book and perhaps read it differently. I guess I am not a fan of the series, but it's ok for that. It was a good read, so good.",
 "<|startoftext|>~`bad~^boring~@**spoile